# Ep 9  POST 방식으로 검색을 구현한 페이지 크롤링
    https://www.bigkinds.or.kr/ 사이트의 검색을 하면 나오는 글들을 크롤링하는 것이 목표

### post 형식 = >
    
    - url(페이지주소 부분)에 get 인자가 확인되지 않을때, = url 이 변경되지 않을때.
    
    - requests 변수 쓸때만 따로 data 사전만 입력하면
      그 이후의 과정은 해당 페이지의 html 을 그대로 보고 하면 된다.!
      
      => 즉 응답을 주고 받을때만 차이가 있지, html 의 구조에는 차이가 없다 ("개발자 도구 기준")



### 페이지 주소 부분에 get 인자가 없을 경우!!

    1) 페이지 주소가 변경되지 않을때 post 형식이 아닐까 의심
    2) 일단 개발자 도구를 눌러 
        -> Network -> 새로고침
        -> (가장상단, or .do 형식 => 정확하지 않다) 있는거 클릭 => 요령껏 주소에 포         함되지 않는 것을 의심해라
        -> Headers 클릭 
        -> General 의 Requests Method 가
            1. post 인지
            2. get 인지 확인해라
            
####              post 방식은 get 인자를 body에 넣어서 보낸다. 
        form data 부분에 보면 get 인자가 있다.

##### +a) 개발자 도구 Network 에 대하여
       - Header 부분은 = HTML 의 <header>안에 잇는 내용을 보기좋게 표시한것
              ex) 인코딩, 타이틀 언어 등등등
       - response 부분은 = HTML 의 구조를 보여주는 것이다.
       
              => 보면 일반적인 get 형태의 페이지의 html 과 유사한 형태가 있다

##### +a) post 형식의 page 크롤링 하는 방법

    1) 일단 기본 url(= "주소창에 뜨는 것")을 지정한다.
    
    2) network 에서 찾은 post 형식의 응답을 찾아서
    
    3) form data 인자를 전부 긁어온다.
    
    4) 위의 인자를 일정한 전처리를 통해 사전형태로 변환한다.
    
    5) requsts.post(기본 url, data = 전처리사전).text 형태로 긁어온다.

In [1]:
import requests
from bs4 import BeautifulSoup

In [ ]:
url_main = "https://www.bigkinds.or.kr/news/newsResult.do"

#### * form data 에서 인자들을 -> 사전형식으로

    1) 문자열로 변환
    2) 띄어쓰기를 기준으로 데이터를 나누느 함수 = "splitlines()"이용
    3) 하나의 데이터에서 ":" 을 기준으로 두개의 인자로 나누는 var.split('기준',?) 이용
    +) 인자가 없는 것은 null 에서 -> none 으로 변경
    4) 나누어진 인자들을 각 각 key 와 value 변수에 넣는다.
    5) 각인자를 빈사전 형태 에 삽입!

In [3]:
lines = '''pageInfo:bksMain
login_chk:null
LOGIN_SN:null
LOGIN_NAME:null
indexName:news
keyword:문재인
byLine:
searchScope:1
searchFtr:1
startDate:2017-10-13
endDate:2018-01-13
sortMethod:date
contentLength:100
providerCode:
categoryCode:
incidentCode:
dateCode:
highlighting:true
sessionUSID:
sessionUUID:test
listMode:
categoryTab:
newsId:
delnewsId:
delquotationtxt:
filterProviderCode:
filterCategoryCode:
filterIncidentCode:
filterDateCode:
filterAnalysisCode:
startNo:1
resultNumber:10
topmenuoff:
resultState:
keywordJson:
keywordFilterJson:
realKeyword:
keywordYn:Y
totalCount:
interval:
quotationKeyword1:
quotationKeyword2:
quotationKeyword3:
printingPage:
searchFromUseYN:N
mainTodayPersonYn:
period:3month
sectionDiv:'''.splitlines()
lines

['pageInfo:bksMain',
 'login_chk:null',
 'LOGIN_SN:null',
 'LOGIN_NAME:null',
 'indexName:news',
 'keyword:문재인',
 'byLine:',
 'searchScope:1',
 'searchFtr:1',
 'startDate:2017-10-13',
 'endDate:2018-01-13',
 'sortMethod:date',
 'contentLength:100',
 'providerCode:',
 'categoryCode:',
 'incidentCode:',
 'dateCode:',
 'highlighting:true',
 'sessionUSID:',
 'sessionUUID:test',
 'listMode:',
 'categoryTab:',
 'newsId:',
 'delnewsId:',
 'delquotationtxt:',
 'filterProviderCode:',
 'filterCategoryCode:',
 'filterIncidentCode:',
 'filterDateCode:',
 'filterAnalysisCode:',
 'startNo:1',
 'resultNumber:10',
 'topmenuoff:',
 'resultState:',
 'keywordJson:',
 'keywordFilterJson:',
 'realKeyword:',
 'keywordYn:Y',
 'totalCount:',
 'interval:',
 'quotationKeyword1:',
 'quotationKeyword2:',
 'quotationKeyword3:',
 'printingPage:',
 'searchFromUseYN:N',
 'mainTodayPersonYn:',
 'period:3month',
 'sectionDiv:']

In [16]:
data_1 = {}

for index in lines:
    key_1, value_1 = index.split(':',1)##### ":"가 한개 있을때만(=뒤에 1은 앞의 기준의 문자가 몇개가 있을때 인가 의미) ":"을 기준으로 나누어라
    if value_1 == "null":
        value_1 = None
    ### 사전형의 값들을 지정하는 방식 주의!
    data_1[key_1] = value_1
    
    
data_1
     

{'LOGIN_NAME': None,
 'LOGIN_SN': None,
 'byLine': '',
 'categoryCode': '',
 'categoryTab': '',
 'contentLength': '100',
 'dateCode': '',
 'delnewsId': '',
 'delquotationtxt': '',
 'endDate': '2018-01-13',
 'filterAnalysisCode': '',
 'filterCategoryCode': '',
 'filterDateCode': '',
 'filterIncidentCode': '',
 'filterProviderCode': '',
 'highlighting': 'true',
 'incidentCode': '',
 'indexName': 'news',
 'interval': '',
 'keyword': '문재인',
 'keywordFilterJson': '',
 'keywordJson': '',
 'keywordYn': 'Y',
 'listMode': '',
 'login_chk': None,
 'mainTodayPersonYn': '',
 'newsId': '',
 'pageInfo': 'bksMain',
 'period': '3month',
 'printingPage': '',
 'providerCode': '',
 'quotationKeyword1': '',
 'quotationKeyword2': '',
 'quotationKeyword3': '',
 'realKeyword': '',
 'resultNumber': '10',
 'resultState': '',
 'searchFromUseYN': 'N',
 'searchFtr': '1',
 'searchScope': '1',
 'sectionDiv': '',
 'sessionUSID': '',
 'sessionUUID': 'test',
 'sortMethod': 'date',
 'startDate': '2017-10-13',
 'startNo

In [17]:
url_main = "https://www.bigkinds.or.kr/news/newsResult.do"
#data = 앞에서 정의한 사전


## 일단 응답이 몇으로 오는지 확인, 
## 200 정도가 와야 정상
response_1 = requests.post(url_main,data=data_1)
response_1

<Response [200]>

In [18]:
html_1 = response_1.text
soup_1 = BeautifulSoup(html_1,"html.parser")
soup_1


<!DOCTYPE html>

<html lang="ko">
<head>
<meta charset="utf-8"/>
<!-- Open Graph Protocol info -->
<meta content="BIG KINDS PRO" property="og:title">
<meta content="website" property="og:type"/>
<meta content="http://www.bigkinds.or.kr/webcont/images/logo_top_main.png" property="og:image"/>
<meta content="http://www.bigkinds.or.kr" property="og:url"/>
<meta content="뉴스빅데이터 분석시스템, 뉴스 속 키워드 관계망, 주요 이슈, 정보원, 이슈 트렌드 분석 정보 제공" property="og:description"/>
<!-- Etc -->
<meta content="IE=edge" http-equiv="X-UA-Compatible">
<meta content="BIG KINDS, big kinds, 빅카인즈, 빅 카인즈, 카인즈, kinds, Kinds" name="keywords"/>
<meta content="뉴스빅데이터 분석시스템, 뉴스 속 키워드 관계망, 주요 이슈, 정보원, 이슈 트렌드 분석 정보 제공" name="description"/>
<meta content="width=device-width, initial-scale=1, maximum-scale=1, user-scalable=no" name="viewport"/>
<title>BigKinds
			
		</title>
<script>
			//브라우저 체크해서 폰트 CSS 구분
			window.onload = function (){
				var agent = navigator.userAgent.toLowerCase();

			    var nowDate = new Date(); // 캐쉬문제로 인한

In [22]:
### 각 기사의 제목만 긁어오기!

for tag in soup_1.select(".resultList .resTxt h3"):
    print(tag.text)

靑 '가상화폐 광풍' 컨트롤타워 되나?

[이슈+] 가상화폐 엇박자 배경은…‘바다이야기’ 트라우마

"내가 누군지 알아?"…허동준 민주당 당협위원장 음주측정 거부 논란

한국당, 北 예술단 우선협의 제의에 "대남전략에 놀아나선 안돼"

남경필 "조조되어 동탁 토벌" 자유한국당 복당 초읽기…이재명 "조조 아닌 여포" 반박

“정부에 북핵부터 해결하라는 건 천지 분간 못하는 소리”

[특파원+] 한국 여당은 후보 넘쳐 고민, 미국 여당은 출마포기 잇따라 난감

홍준표 "우리가 유치한 평창, 남북 정치쇼 이용"

사면·복권된 용산참사 생존자들 “MB, 경찰 수뇌부, 개발업자가 책임져야”

예고된 이별, ‘국민의당’ 간판은 누가 지킬까



In [23]:
#####위의 결과에서 띄어쓰기가 (계행이 있는 것 같다. 없애자 ) => strip() 함수 이용
for tag in soup_1.select(".resultList .resTxt h3"):
    print(tag.text.strip())

靑 '가상화폐 광풍' 컨트롤타워 되나?
[이슈+] 가상화폐 엇박자 배경은…‘바다이야기’ 트라우마
"내가 누군지 알아?"…허동준 민주당 당협위원장 음주측정 거부 논란
한국당, 北 예술단 우선협의 제의에 "대남전략에 놀아나선 안돼"
남경필 "조조되어 동탁 토벌" 자유한국당 복당 초읽기…이재명 "조조 아닌 여포" 반박
“정부에 북핵부터 해결하라는 건 천지 분간 못하는 소리”
[특파원+] 한국 여당은 후보 넘쳐 고민, 미국 여당은 출마포기 잇따라 난감
홍준표 "우리가 유치한 평창, 남북 정치쇼 이용"
사면·복권된 용산참사 생존자들 “MB, 경찰 수뇌부, 개발업자가 책임져야”
예고된 이별, ‘국민의당’ 간판은 누가 지킬까


##### +a) 팝업창이 뜨면서  새로운 창으로 연결되는 페이지 크롤링

    1) network 에서 기존에 있던 것을 지운다 좌측상단 (clear 를 클릭)
    2) 팝업창을 누룬다.
    3) 새롭게 뜨는 응답을 클릭한다.
        제일 심플한 경우는 눌렀을때 따로 전처리 필요없는 사전형태의 데이터
        
         "이제 해당 url로 연결되는 href 주소를 따내는 것이 핵심이다. " -> 일반적으로 html(element) 구조안에 id 등에 표시되어 있다.
     => 즉 해당 href의 특정(숫자가) 표시되어 있는 부분은 html 에서 찾아내는 것이 핵심이다.

In [24]:
##### 해당 팝업창의 주소는 h3 id 에 있는 것 같다.
for tag in soup_1.select(".resultList .resTxt h3"):
    
    doc_url = "https://www.bigkinds.or.kr/news/detailView.do?docId=01100701.20180113180258001&returnCnt=1&sectionDiv=1000"
    print(tag["id"])

news_08100401.20180113201802001
news_01100701.20180113193955001
news_01100701.20180113180258001
news_01200201.20180113171536001
news_01100701.20180113162906001
news_01100101.20180113160908001
news_01100701.20180113160154001
news_08100401.20180113154803001
news_01100101.20180113153027001
news_01100101.20180113152514001


In [25]:
#### 위에서 new_ 부분을 제거해야된다.  -> "replace("바꾸고 싶은대상", "바꾸고 싶은 형태")"
for tag in soup_1.select(".resultList .resTxt h3"):
    print(tag["id"].replace("news_",""))

08100401.20180113201802001
01100701.20180113193955001
01100701.20180113180258001
01200201.20180113171536001
01100701.20180113162906001
01100101.20180113160908001
01100701.20180113160154001
08100401.20180113154803001
01100101.20180113153027001
01100101.20180113152514001


In [28]:
##### 이제 팝업창 주소와 제목을 같이 print 해보자
for tag in soup_1.select(".resultList .resTxt h3"):
    
    doc_id = tag["id"].replace("news_","")
    doc_url = "https://www.bigkinds.or.kr/news/detailView.do?docId={}&returnCnt=1&sectionDiv=1000".format(doc_id)#### 문자열 포멧팅!!!
    print(tag.text.strip(),doc_url)

靑 '가상화폐 광풍' 컨트롤타워 되나? https://www.bigkinds.or.kr/news/detailView.do?docId=08100401.20180113201802001&returnCnt=1&sectionDiv=1000
[이슈+] 가상화폐 엇박자 배경은…‘바다이야기’ 트라우마 https://www.bigkinds.or.kr/news/detailView.do?docId=01100701.20180113193955001&returnCnt=1&sectionDiv=1000
"내가 누군지 알아?"…허동준 민주당 당협위원장 음주측정 거부 논란 https://www.bigkinds.or.kr/news/detailView.do?docId=01100701.20180113180258001&returnCnt=1&sectionDiv=1000
한국당, 北 예술단 우선협의 제의에 "대남전략에 놀아나선 안돼" https://www.bigkinds.or.kr/news/detailView.do?docId=01200201.20180113171536001&returnCnt=1&sectionDiv=1000
남경필 "조조되어 동탁 토벌" 자유한국당 복당 초읽기…이재명 "조조 아닌 여포" 반박 https://www.bigkinds.or.kr/news/detailView.do?docId=01100701.20180113162906001&returnCnt=1&sectionDiv=1000
“정부에 북핵부터 해결하라는 건 천지 분간 못하는 소리” https://www.bigkinds.or.kr/news/detailView.do?docId=01100101.20180113160908001&returnCnt=1&sectionDiv=1000
[특파원+] 한국 여당은 후보 넘쳐 고민, 미국 여당은 출마포기 잇따라 난감 https://www.bigkinds.or.kr/news/detailView.do?docId=01100701.20180113160154001&returnCnt=1&sectionDiv=1000


### 정리

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
lines = '''pageInfo:bksMain
login_chk:null
LOGIN_SN:null
LOGIN_NAME:null
indexName:news
keyword:문재인
byLine:
searchScope:1
searchFtr:1
startDate:2017-10-13
endDate:2018-01-13
sortMethod:date
contentLength:100
providerCode:
categoryCode:
incidentCode:
dateCode:
highlighting:true
sessionUSID:
sessionUUID:test
listMode:
categoryTab:
newsId:
delnewsId:
delquotationtxt:
filterProviderCode:
filterCategoryCode:
filterIncidentCode:
filterDateCode:
filterAnalysisCode:
startNo:1
resultNumber:10
topmenuoff:
resultState:
keywordJson:
keywordFilterJson:
realKeyword:
keywordYn:Y
totalCount:
interval:
quotationKeyword1:
quotationKeyword2:
quotationKeyword3:
printingPage:
searchFromUseYN:N
mainTodayPersonYn:
period:3month
sectionDiv:'''.splitlines()
lines

In [ ]:
data_1 = {}

for index in lines:
    key_1, value_1 = index.split(':',1)
    if value_1 == "null":
        value_1 = None
    data_1[key_1] = value_1

url_main = "https://www.bigkinds.or.kr/news/newsResult.do"

### 응답 요청하기
response_1 = requests.post(url_main,data=data_1)
response_1

In [ ]:
html_1 = response_1.text
soup_1 = BeautifulSoup(html_1,"html.parser")

##### 긁어오기
for tag in soup_1.select(".resultList .resTxt h3"):
    
    doc_id = tag["id"].replace("news_","")
    doc_url = "https://www.bigkinds.or.kr/news/detailView.do?docId={}&returnCnt=1&sectionDiv=1000".format(doc_id)#### 문자열 포멧팅!!!
    print(tag.text.strip(),doc_url)